In [1]:
import numpy as np

In [ ]:
np.sign()

In [ ]:
        def absolute_sdm(obs_c, mod_c ,sce_c, *args, **kwargs):
            """
            apply absolute scaled distribution mapping to all scenario cubes
            assuming a normal distributed parameter

            Args:

            * obs_cube (:class:`iris.cube.Cube`):
                the observational data

            * mod_cube (:class:`iris.cube.Cube`):
                the model data at the reference period

            * sce_cubes (:class:`iris.cube.CubeList`):
                the scenario data that shall be corrected

            Kwargs:

            * cdf_threshold (float):
                limit of the cdf-values (default: .99999)
            """
            
            from scipy.stats import norm
            from scipy.signal import detrend
            
            # extract keywords
            cdf_threshold = kwargs.get('cdf_threshold', .99999)
            
            # STEP 1
            # original series their length
            data = {'obs': obs_c.copy(),
                    'mod': mod_c.copy(),
                    'sce': sce_c.copy()}
            # length of the series
            lens = {key: len(data[key]) for key in data}
            # mean of the series
            means = {key: data[key].mean() for key in data}
            # detrend the data
            detrended = key: detrend(data[key]) for key in data}

            # STEP 2
            # fit normal distribution
            pars = {key: norm.fit(detrended[key], floc=0) for key in detrended}
            # corresponding cdf values
            cdf = {}
            for key in detrended:
                cdf_aux = norm.cdf(np.sort(detrended[key]), *pars[key])
                cdf[key] = np.maximum(np.minimum(cdf_aux, cdf_threshold), 1 - cdf_threshold)
                del cdf_aux
            
            # STEP 3
            # scaling factor between the historic and future model
            SFa = (norm.ppf(cdf['sce'], *pars['sce']) - norm.ppf(cdf['sce'], *pars['mod']) * \
                   pars['obs'][-1] / pars['mod'][-1]
            
            # STEP 4
            # linearly interpolate cdf-values for 'obs' and 'mod' to the length of the scenario
            for key in ['obs', 'mod']:
                cdf[key + '_int'] = np.interp(np.linspace(1, lens[key], lens['sce']),
                                              np.linspace(1, lens[key], lens[key]),
                                              cdf[key])
            # split the tails of the cdfs around the center
            cdf_shift = {key: cdf[key] - .5 for key in ['obs_int', 'mod_int', 'sce']}
            # recurrence intervals
            RI = {key: 1. / (.5 - np.abs(cdf_shift[key])) for key in cdf_shift}
            
            # STEP 5
            # find RIscaled for the raw future model
            RIscaled = np.maximum(1., RI['obs_int'] * RI['sce'] / RI['mod_int'])
            # adapt the observation cdfs
#            adapted_cdf = .5 + np.sign(cdf_shift['obs_int']) * np.abs(.5 - 1 / RIscaled)
            adapted_cdf = np.sign(cdf_shift['obs_int']) * (1. - 1. / (RI['obs_int'] * RI['sce'] / RI['mod_int']))
            adapted_cdf[adapted_cdf < 0] += 1.
            adapted_cdf = np.maximum(np.minimum(adapted_cdf, cdf_threshold), 1 - cdf_threshold)
            
            # STEP 6
            # initial array of bias corrected values
            xvals = norm.ppf(np.sort(adapted_cdf), *pars['obs']) + SFa
            xvals -= xvals.mean()
            xvals += means['obs'] + means['sce'] - means['mod']
            
            # STEP 7
            # reinsert the bias corrected values
            correction = np.zeros(lens['sce'])
            sce_argsort = np.argsort(detrended['sce'])
            correction[sce_argsort] = xvals
            # add back the trend of the future model
            sce_diff = data['sce'] - detrended['sce']
            correction += sce_diff - means['sce']
            
            return correction